In [ ]:
#Initialize dependencies
import tensorflow as tf
import os
BASE_FOLDER = '/Users/oblivisheee/Documents/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)
%store -r
from modules.transformer_custom import Transformer
from modules.regularization import RegularizedDenseLayer
from modules.session import SessionCreator
#from modules.data_preprocessing import Data


In [ ]:
from modules.data_preprocess import DataGenerator

train_dir = 'path/to/train/data'
valid_dir = 'path/to/valid/data'

data_gen = DataGenerator(train_dir, valid_dir)
train_data, valid_data = data_gen.generate()

train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
valid_dataset = tf.data.Dataset.from_tensor_slices(valid_data)



# Config training

In [ ]:
# Configure the Transformer layer

embedding_layer = tf.keras.layers.Embedding(8500, 512)
regularized_layer = RegularizedDenseLayer(512)

model = Transformer(num_layers=6, d_model=512, num_heads=8, dff=2048,
                          input_vocab_size=8500, target_vocab_size=8500, pe_input=10000,
                          pe_target=10000, rate=0.1, embedding=embedding_layer, regularized_layer=regularized_layer)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='auto')
optimizer = tf.keras.optimizers.Adam(name='Adam',learning_rate=0.001,
                                     epsilon=1e-8, amsgrad=True,
                                     beta_1=0.9, beta_2=0.999)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'), 
           tf.keras.metrics.MeanSquaredError(name='mean_squared_error'), 
           tf.keras.metrics.Precision(thresholds=0.5, name='precision'),]


model.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)

# Start training

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
epochs = 10
model.fit_model(inp_dataset=train_dataset, tar_dataset=train_dataset, valid_inp_dataset=valid_dataset, valid_tar_dataset=valid_dataset, epochs=epochs,
                         session_name=session_name,
                         batch_size=64, gradient_accumulation_steps=5,
                         shuffle=True,
                         save_model_each_epoch=True,
                         model_name='novelsdreamer-ru-t4m')